### Import all packages

In [1]:
import sys
sys.path.append("..") 

In [2]:
import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
from pathlib import Path
import os
import torch.cuda
import albumentations as albu
import segmentation_models_pytorch as smp
from  segmentation_models_pytorch.utils.base import Metric
from segmentation_models_pytorch.base.modules import Activation
from collections import defaultdict
import nibabel as nib
from tqdm import tqdm
from Utils.dataset_utils import *
from Utils.prediction_utils import *


In [3]:
from torchsummary import summary
from torch.utils.data import DataLoader
from torch.utils.data import Dataset as BaseDataset

In [4]:
VOLUMES_BASE_PATH=Path(r'G:\Projects and Work\Mouse Heart Segmentation\Mice_CT_Dataset')
LABELS_TO_KEEP=[1]
SUBJECTS=os.listdir(VOLUMES_BASE_PATH/'volumes')
# SUBJECTS=list(filter(lambda x: x.find("LAD") != -1,SUBJECTS))
# print(SUBJECTS)

In [5]:
os.listdir(VOLUMES_BASE_PATH)

['Axials', 'labels', 'volumes']

In [6]:
WIDTH=512
HEIGHT=512

In [7]:
def create_folder(path):
    if os.path.exists(path)==False:
        os.mkdir(path)

In [8]:
os.listdir(r'G:\Projects and Work\Mouse Heart Segmentation\Trained Weights - 13 Mice Volumes')

['logs',
 'Unet_efficientnet-b4',
 'Unet_efficientnet-b4_WHS_Fine_Tuned',
 'Unet_resnet101',
 'Unet_resnet101_WHS_Fine_Tuned',
 'Unet_se_resnext50_32x4d',
 'Unet_se_resnext50_32x4d_WHS_Fine_Tuned']

In [9]:
# Model to choose
ENCODERS = ['efficientnet-b4','resnet101','se_resnext50_32x4d']
ENCODER_WEIGHTS = 'imagenet'
DEVICE = 'cuda'
MODEL_NAME='Unet'

In [10]:
# function to save numpy volume array to .nii


In [11]:
# Run a model on all volumes
for ENCODER in ENCODERS:
    BEST_WEIGHTS=fr'G:\Projects and Work\Mouse Heart Segmentation\Trained Weights - 13 Mice Volumes\Unet_{ENCODER}\best_score.pt'
    OUTPUT_PATH=Path(fr'G:\Projects and Work\Mouse Heart Segmentation\Trained Weights - 13 Mice Volumes\Unet_{ENCODER}\Outputs New')
    create_folder(OUTPUT_PATH)
    # Load model
    preprocessing_fn = smp.encoders.get_preprocessing_fn(ENCODER, ENCODER_WEIGHTS)
    best_model = torch.load(BEST_WEIGHTS)

    for i in range(len(SUBJECTS)):
        print(SUBJECTS[i])
        subject_name = SUBJECTS[i]
        subject_path = os.path.join(VOLUMES_BASE_PATH,'volumes',subject_name,f'{subject_name}_volume.nii') # volume path
        if os.path.exists(subject_path) is False: #if .nii file of volume ct not present then skip
            continue
        subject_label_path = os.path.join(VOLUMES_BASE_PATH,'labels',subject_name ,f'{subject_name}_label.nii') #volume label
        if os.path.exists(subject_label_path)==False: # checking if extension is .nii or .nii.gz 
            subject_label_path = os.path.join(VOLUMES_BASE_PATH,'labels',subject_name,f'{subject_name}_label.nii.gz')   
        if os.path.exists(os.path.join(OUTPUT_PATH,subject_name)):
            continue
        create_folder(os.path.join(OUTPUT_PATH,subject_name)) # create output folder
        volume, volume_gt_mask,affine = load_case(subject_path,subject_label_path) # Load volume with its affine and mask
        volume_pred_mask=predict_volume(best_model,volume,True,preprocessing_fn)   # Predict volume
        volume_pred_mask=np.round(volume_pred_mask)
        save_mask_nii(volume_pred_mask,affine,os.path.join(OUTPUT_PATH,subject_name,'prediction.nii.gz'))    #save predicted volume
        if volume_gt_mask is not None:     # save grouth truth .nii file also for convinience
            save_mask_nii(volume_gt_mask,affine,os.path.join(OUTPUT_PATH,subject_name,'ground_truth.nii.gz'))

C57-#10-MicroCT-020421 - Cardiac CT_110016-diastole
C57-#10-MicroCT-020421 - Cardiac CT_110016-systole
C57-#6-MicroCT-020421 - Cardiac CT_110734-diastole
C57-#6-MicroCT-020421 - Cardiac CT_110734-systole
C57-#8-MicroCT-020421 - Cardiac CT_104450-diastole
C57-#8-MicroCT-020421 - Cardiac CT_104450-systole
C57-#9-MicroCT-020421 - Cardiac CT_105254-diastole
C57-#9-MicroCT-020421 - Cardiac CT_105254-systole
C57-UmaControl#1-MicroCT-070120 - Cardiac CT_115114-diastole
G:\Projects and Work\Mouse Heart Segmentation\Mice_CT_Dataset\labels\C57-UmaControl#1-MicroCT-070120 - Cardiac CT_115114-diastole\C57-UmaControl#1-MicroCT-070120 - Cardiac CT_115114-diastole_label.nii.gz


100%|███████████████████████████████████████████████████████████████████████| 512/512 [00:35<00:00, 14.53it/s]


C57-UmaControl#1-MicroCT-070120 - Cardiac CT_115114-systole
C57-UmaControl#2-MicroCT-070120 - Cardiac CT_115916-diastole
C57-UmaControl#2-MicroCT-070120 - Cardiac CT_115916-systole
C57-UmaControl#3-MicroCT-070120 - Cardiac CT_121937-diastole
C57-UmaControl#3-MicroCT-070120 - Cardiac CT_121937-systole
LAD-#11-MicroCT-050321 - Cardiac CT_133143-diastole
LAD-#11-MicroCT-050321 - Cardiac CT_133143-systole
LAD-#12-MicroCT-050321 - Cardiac CT_133905-diastole
LAD-#12-MicroCT-050321 - Cardiac CT_133905-systole
LAD-#6-MicroCT-050321 - Cardiac CT_125739-diastole
LAD-#6-MicroCT-050321 - Cardiac CT_125739-systole
LAD-Uma#1-MicroCT-030519 - Cardiac CT_133621-diastole
LAD-Uma#1-MicroCT-030519 - Cardiac CT_133621-systole
LAD-Uma#5-MicroCT-030519 - Cardiac CT_134513-diastole
LAD-Uma#5-MicroCT-030519 - Cardiac CT_134513-systole
TAC-#3-MicroCT-050321 - Cardiac CT_115640-diastole
TAC-#3-MicroCT-050321 - Cardiac CT_115640-systole
TAC-#5-MicroCT-050321 - Cardiac CT_120528-diastole
TAC-#5-MicroCT-050321 - C

In [12]:
volume_pred_mask.shape

(512, 512, 512)

In [3]:
visualize()

NameError: name 'visualize' is not defined